## Task: Predict weekly sales of a walmart store using linear regression

You are provided with a dataset about Walmart's sales in a CSV file.

This is the historical data that covers sales from 2010-02-05 to 2012-11-01, in the file Walmart_Store_sales. Within this file you will find the following fields:

1. Store - the store number
2. Date - the week of sales
3. Weekly_Sales - sales for the given store
4. Holiday_Flag - whether the week is a special holiday week: 1 – Holiday week 0 – Non-holiday week
5. Temperature - Temperature on the day of sale
6. Fuel_Price - Cost of fuel in the region
7. CPI – Prevailing consumer price index
8. Unemployment - Prevailing unemployment rate

Note:

You might find it helpful to include information about the kind of holiday week it is. This is something you'll need to add through data processing.
You can use this information about the holiday weeks.

Holiday Events:
1. Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
2. Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
3. Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
4. Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

### Tasks:
- Preprocess the data:
    - Remove any column which might not be useful
    - Add holiday type information
    - Normalize the data
- Split the data randomly into training and testing sets (80:20 split)
- Build and train a linear regression model. You can use any framerwork for this (numpy, sklearn or pytorch)
- Evaluate the trained model on test data. Report Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE).
- Visualize model performance using graphs.

In [ ]:
from IPython.display import clear_output

In [ ]:
# Incase you run this notebook outside colab (where the libraries aren't already pre-installed)

%pip install gdown==4.5
%pip install pandas
%pip install numpy

clear_output()

In [ ]:
# Download the CSV file.
!gdown 1UjuHkxE8VSQOdklibKqZ72qodJLBynEv

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_df = pd.read_csv('Walmart.csv')

In [ ]:
data_df.head()

In [ ]:
data_y = data_df['Weekly_Sales'] # target
data_x = data_df.drop(['Weekly_Sales'], axis=1) # input features

In [ ]:
data_x.head()

In [ ]:
data_y

In [ ]:
desc = data_x.describe()
desc

In [ ]:
# Normalize features
own_df= data_x[["Store", "Holiday_Flag", "Temperature", "Fuel_Price", "CPI", "Unemployment"]].apply(lambda x: (x-x.min())/(x.max()-x.min()))

own_df["Date"] = data_df["Date"]

# Sanity check if date was added
#print(own_df)

In [ ]:
y=data_y
y_max = np.max(y)
y_min = np.min(y)
y = (y-y_min)/(y_max-y_min)

# Sanity check
print(min(y), max(y))

# Doing it on my own (numpy)

In [ ]:
# 1 – Remove columns that are not of interest
#     A: I would remove the store column, since we want to predict overall sales of walmart and not drill down for specific loactions

own_df = own_df.drop("Store", axis=1, inplace=False) # axis=1 => delete column, inplace=false => not edit on original data

print(own_df)

In [ ]:
# 2 – Add holiday information

# Check for data type
print("Data-Type before casting: ", type(own_df["Date"].loc[0])) 

# Cast to right datetime format instead of string
own_df["Date"] = pd.to_datetime(own_df["Date"], format="%d-%m-%Y")

# Sanity check
print("Data-Type after casting: ", type(own_df["Date"].loc[0])) 

# Dictionary of holiday dates and types
holiday_dates = {
    "2010-02-12": "Super Bowl",
    "2011-02-11": "Super Bowl",
    "2012-02-10": "Super Bowl",
    "2010-09-10": "Labor Day",
    "2011-09-09": "Labor Day",
    "2012-09-07": "Labor Day",
    "2010-11-26": "Thanksgiving",
    "2011-11-25": "Thanksgiving",
    "2012-11-23": "Thanksgiving",
    "2010-12-31": "Christmas",
    "2011-12-30": "Christmas",
    "2012-12-28": "Christmas"
}

# Add a new column "Holiday_Type" to the DataFrame
own_df["Holiday_Type"] = own_df["Date"].apply(lambda x: holiday_dates.get(x.strftime("%Y-%m-%d"), "None"))

# Check if holidays get assigned
#print(set(own_df["Holiday_Type"]))

In [ ]:
# 3 – One-hot encode "Holiday_Type"

# => adds new columns for every holiday and sets the respective entry to 1 if holiday applies other option is 0
own_df = pd.get_dummies(own_df, columns=['Holiday_Type'], drop_first=True)

own_df.head()

In [ ]:
# 4 – Split train and test data 
from sklearn.model_selection import train_test_split


# Create feature matrix, but without "Weekly Sales" (since it is the target) and Date since it is not relevant.
X = own_df.drop(columns=["Date"])

# Create target variable vector (just assign the one from above for sanity)
y = y

# random_state is seed to have reproducibility 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

# Sanity check to see distribution
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

In [ ]:
# 5 – Build regression model

# Add a column of ones to X_train and X_test for the intercept term (bias or called c in normal mx-function) 
#  This step also casts dataframe to numpy array
ones_train = np.ones((X_train.shape[0], 1))
X_train = np.concatenate([ones_train, X_train], axis=1)

ones_test = np.ones((X_test.shape[0], 1))
X_test = np.concatenate([ones_test, X_test], axis=1)


# Additionally reshape targets to ensure correct dimensions for the matrix operations (multiplication)
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

print(X_train.shape)
print(y_train.shape)

In [ ]:
# 6 – Train the model, using the normal equation

theta = np.linalg.inv(X_train.T @ X_train) @ (X_train.T @ y_train)

print(theta.shape)
print(theta)

In [ ]:
# 7 – Predict targets with trained model with test data

# Predict
y_pred = X_test @ theta

# Calculate Mean Absolute Error (MAE)
mae_own = np.mean(np.abs(y_pred - y_test.flatten()))

# Calculate Mean Squared Error (MSE)
mse_own = np.mean((y_pred - y_test) ** 2)

# Print MAE and MSE
print("Own model MAE: ", mae_own)
print("Own model MSE: ", mse_own)

In [ ]:
# Calculate residuals
residuals_own = y_test - y_pred

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Visualization

In [ ]:
# Create a scatter plot using Plotly
fig = make_subplots()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=np.arange(len(y_test)),
    y=y_test.flatten(),
    mode='markers',
    marker=dict(color='blue'),
    name='Actual Values',
    hovertemplate='Index: %{x}<br>Actual Sales: %{y}<extra></extra>'
))

# Add trace for predicted values
fig.add_trace(go.Scatter(
    x=np.arange(len(y_pred)),
    y=y_pred.flatten(),
    mode='markers',
    marker=dict(color='red'),
    name='Predicted Values – Own Method',
    hovertemplate='Index: %{x}<br>Predicted Sales: %{y}<extra></extra>'
))

# Update layout
fig.update_layout(
    title='Actual vs. Predicted Weekly Sales',
    xaxis_title='Index',
    yaxis_title='Weekly Sales',
    showlegend=True
)

fig.show()


# Why do the sales number look so wierd?

### Because the sales prediction model got normalized input data. Therefor it adapted to the normalized numbers.


In [ ]:
# Create interactive scatter plot for residuals
fig_residuals = go.Figure()

# Add trace for residuals of custom model
fig_residuals.add_trace(go.Scatter(
    x=np.arange(len(y_pred)),
    y=residuals_own.flatten(),
    mode='markers',
    marker=dict(color='blue'),
    name='Residuals (Custom Model)',
    hovertemplate='Index: %{x}<br>Residual: %{y}<extra></extra>',
    hoverinfo='text'
))

# Update layout
fig_residuals.update_layout(
    title='Residuals Plot',
    xaxis_title='Index',
    yaxis_title='Residuals',
    showlegend=True
)

fig_residuals.show()

In [ ]:
# Interactive Distribution of Residuals Plot
fig_dist_residuals = go.Figure()

# Add histogram for residuals
fig_dist_residuals.add_trace(go.Histogram(
    x=residuals_own.flatten(),
    nbinsx=30,
    marker=dict(color='blue'),
    name='Residuals Distribution',
    hovertemplate='Residuals: %{x}<br>Frequency: %{y}<extra></extra>'
))

# Update layout
fig_dist_residuals.update_layout(
    title='Distribution of Residuals',
    xaxis_title='Residuals',
    yaxis_title='Frequency',
    showlegend=True
)

fig_dist_residuals.show()

# Solving it with SK-LEARN

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Train a model using sklearn -> LinearRegression model
model = LinearRegression()
model.fit(X_train, y_train)
y_pred_sklearn = model.predict(X_test)

In [ ]:
# Calculate residuals for sklearn try
residuals_sklearn = y_test - y_pred_sklearn

# Calculate mae, mse for sklearn try
mae_sklearn = np.mean(np.abs(residuals_sklearn))
mse_sklearn = np.mean(residuals_sklearn ** 2)

#print(mae_sklearn)
#print(mse_sklearn)

# Compare the two results

In [ ]:
print("MAE own method", mae_own)
print("MSE own method", mse_own)
print("----")
print("MAE sklearn method", mae_sklearn)
print("MSE slearn method", mse_sklearn)

print("\nInterpretation: \n")
print("Lower MAE means that, on average,the model's predictions are closer to the actual values.\n")
print("Lower MSE indicates that the model has fewer large errors, which is to consider if you have a dataset with many outliers. These outliers than dont have such a big impact")

print("\n-> sklearn model is preferable since it has a lower mae as well as mse")

In [ ]:
# Create a scatter plot using Plotly
fig = make_subplots()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=np.arange(len(y_test)),
    y=y_test.flatten(),
    mode='markers',
    marker=dict(color='blue'),
    name='Actual Values',
    hovertemplate='Index: %{x}<br>Actual Sales: %{y}<extra></extra>'
))

# Add trace for predicted values (own method)
fig.add_trace(go.Scatter(
    x=np.arange(len(y_pred)),
    y=y_pred.flatten(),
    mode='markers',
    marker=dict(color='red'),
    name='Predicted Values – Own Method',
    hovertemplate='Index: %{x}<br>Predicted Sales: %{y}<extra></extra>'
))


# Add trace for predicted values (sklearn method)
fig.add_trace(go.Scatter(
    x=np.arange(len(y_pred_sklearn)),
    y=y_pred_sklearn.flatten(),
    mode='markers',
    marker=dict(color='green'),
    name='Predicted Values – sklearn',
    hovertemplate='Index: %{x}<br>Predicted Sales: %{y}<extra></extra>'
))


# Update layout
fig.update_layout(
    title='Actual vs. Predicted Weekly Sales',
    xaxis_title='Index',
    yaxis_title='Weekly Sales',
    showlegend=True
)

fig.show()

In [ ]:
# Create interactive scatter plot for residuals
fig_residuals = go.Figure()

# Add trace for residuals of custom model
fig_residuals.add_trace(go.Scatter(
    x=np.arange(len(y_pred)),
    y=residuals_own.flatten(),
    mode='markers',
    marker=dict(color='blue'),
    name='Residuals (Custom Model)',
    hovertemplate='Index: %{x}<br>Residual: %{y}<extra></extra>',
    hoverinfo='text'
))

# Add trace for residuals of sklearn model
fig_residuals.add_trace(go.Scatter(
    x=np.arange(len(y_pred)),
    y=residuals_sklearn.flatten(),
    mode='markers',
    marker=dict(color='red'),
    name='Residuals (Sklearn Model)',
    hovertemplate='Index: %{x}<br>Residual: %{y}<extra></extra>',
    hoverinfo='text'
))


# Update layout
fig_residuals.update_layout(
    title='Residuals Plot',
    xaxis_title='Index',
    yaxis_title='Residuals',
    showlegend=True
)

fig_residuals.show()

In [ ]:
coefficients = model.coef_
intercept = model.intercept_

# Print results
print(f"Coefficients (theta values): \n{coefficients.reshape(-1,1)}")

In [ ]:
print(theta)